# Finetune a BERT Text Classifier with LightningTrainer

This is an advanced example for LightningTrainer, which demonstrates how to use LightningTrainer with Dataset.

If you just want to quickly convert your existing PyTorch Lightning scripts into Ray AIR, you can refer to this starter example: Train a Pytorch Lightning Image Classifier.

Source: https://docs.ray.io/en/latest/train/examples/lightning/lightning_cola_advanced.html

In [1]:
import ray
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import numpy as np

## Pre-process CoLA Dataset

In [2]:
dataset = load_dataset("glue", "cola")
metric = load_metric("glue", "cola")

ray_datasets = ray.data.from_huggingface(dataset)

/tmp/ipykernel_18727/2004070181.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "cola")
2023-09-07 08:56:16,695	WARNING read_api.py:1794 -- You provided a Huggingface DatasetDict which contains multiple datasets. The output of `from_huggingface` is a dictionary of Ray Datasets. To convert just a single Huggingface Dataset to a Ray Dataset, specify a split. For example, `ray.data.from_huggingface(my_dataset_dictionary['train'])`. Available splits are ['train', 'validation', 'test'].
2023-09-07 08:56:16,707	INFO worker.py:1431 -- Connecting to existing Ray cluster at address: 192.168.33.188:6379...
2023-09-07 08:56:16,711	INFO worker.py:1612 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [3]:
from ray.data.preprocessors import BatchMapper

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_sentence(batch):
    encoded_sent = tokenizer(
        batch["sentence"].tolist(),
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    batch["input_ids"] = encoded_sent["input_ids"].numpy()
    batch["attention_mask"] = encoded_sent["attention_mask"].numpy()
    batch["label"] = np.array(batch["label"])
    batch.pop("sentence")
    return batch


preprocessor = BatchMapper(tokenize_sentence, batch_format="numpy")

## Define a PyTorch Lightning Model

In [4]:
class SentimentModel(pl.LightningModule):
    def __init__(self, lr=2e-5, eps=1e-8):
        super().__init__()
        self.lr = lr
        self.eps = eps # epsilon
        self.num_classes = 2
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-cased", num_labels=self.num_classes
        )
        self.metric = load_metric("glue", "cola")
        self.predictions = []
        self.references = []

    def forward(self, batch):
        input_ids, attention_mask = batch["input_ids"], batch["attention_mask"]
        outputs = self.model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

    def training_step(self, batch, batch_idx):
        labels = batch["label"]
        logits = self.forward(batch)
        loss = F.cross_entropy(logits.view(-1, self.num_classes), labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch["label"]
        logits = self.forward(batch)
        preds = torch.argmax(logits, dim=1)
        self.predictions.append(preds)
        self.references.append(labels)

    def on_validation_epoch_end(self):
        predictions = torch.concat(self.predictions).view(-1)
        references = torch.concat(self.references).view(-1)
        matthews_correlation = self.metric.compute(
            predictions=predictions, references=references
        )

        # self.metric.compute() returns a dictionary:
        # e.g. {"matthews_correlation": 0.53}
        self.log_dict(matthews_correlation, sync_dist=True)
        self.predictions.clear()
        self.references.clear()

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, eps=self.eps)

## Configure your LightningTrainer

In [5]:
from ray.train.lightning import LightningTrainer, LightningConfigBuilder
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig

# Define the configs for LightningTrainer
lightning_config = (
    LightningConfigBuilder()
    .module(cls=SentimentModel, lr=1e-5, eps=1e-8)
    .trainer(max_epochs=5, accelerator="gpu")
    .checkpointing(save_on_train_epoch_end=False)
    .build()
)

In [6]:
# Save AIR checkpoints according to the performance on validation set
run_config = RunConfig(
    name="ptl-sent-classification",
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="matthews_correlation",
        checkpoint_score_order="max",
    ),
)

# Scale the DDP training workload across 4 GPUs
# You can change this config based on your compute resources.
scaling_config = ScalingConfig(
    num_workers=1, use_gpu=True, resources_per_worker={"CPU": 15, "GPU": 1}
)

## Fine-tune the model with LightningTrainer

In [7]:
trainer = LightningTrainer(
    lightning_config=lightning_config,
    run_config=run_config,
    scaling_config=scaling_config,
    datasets={"train": ray_datasets["train"], "val": ray_datasets["validation"]},
    datasets_iter_config={"batch_size": 32},
    preprocessor=preprocessor,
)
result = trainer.fit()

(LightningTrainer pid=18987) The `preprocessor` arg to Trainer is deprecated. Apply preprocessor transformations ahead of time by calling `preprocessor.transform(ds)`. Support for the preprocessor arg will be dropped in a future release.
(LightningTrainer pid=18987) Starting distributed worker processes: ['19030 (192.168.33.188)']
(RayTrainWorker pid=19030) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=19030) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
(RayTrainWorker pid=19030) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(RayTrainWorker pid=19030) /tmp/ipykernel_18727/3457447719.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https:/

(pid=19030) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=19030) - RandomizeBlockOrder: 0 active, 0 queued, 0.0 MiB objects, 0 output 1:   0%|          | 0/1 [00:0…

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  4.79it/s]


(RayTrainWorker pid=19030) /home/mpp/.conda/envs/ray-torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(RayTrainWorker pid=19030)   rank_zero_warn(
(RayTrainWorker pid=19030) Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(BatchMapper._transform_numpy)] -> AllToAllOperator[RandomizeBlockOrder]
(RayTrainWorker pid=19030) Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(RayTrainWorker pid=19030) Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_opti

...

2023-09-07 09:09:37,968	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Epoch 3: : 400it [02:14,  2.98it/s, v_num=0]
Epoch 3: : 401it [02:14,  2.98it/s, v_num=0]
Epoch 3: : 402it [02:14,  2.98it/s, v_num=0]
Epoch 3: : 403it [02:15,  2.98it/s, v_num=0]
Epoch 3: : 404it [02:15,  2.99it/s, v_num=0]
Epoch 3: : 405it [02:15,  2.99it/s, v_num=0]
Epoch 3: : 406it [02:15,  2.99it/s, v_num=0]
Epoch 3: : 407it [02:16,  2.99it/s, v_num=0]
Epoch 3: : 408it [02:16,  2.99it/s, v_num=0]
Epoch 3: : 409it [02:16,  2.99it/s, v_num=0]
Epoch 3: : 410it [02:17,  2.99it/s, v_num=0]
Epoch 3: : 411it [02:17,  2.99it/s, v_num=0]
Epoch 3: : 412it [02:17,  2.99it/s, v_num=0]
Epoch 3: : 413it [02:17,  3.00it/s, v_num=0]
Epoch 3: : 414it [02:18,  3.00it/s, v_num=0]
Epoch 3: : 415it [02:18,  3.00it/s, v_num=0]
Epoch 3: : 416it [02:18,  3.00it/s, v_num=0]
Epoch 3: : 417it [02:19,  3.00it/s, v_num=0]
Epoch 3: : 418it [02:19,  3.00it/s, v_num=0]
Epoch 3: : 419it [02:19,  3.00it/s, v_num=0]
Epoch 3: : 420it [02:19,  3.00it/s, v_num=0]
Epoch 3: : 421it [02:20,  3.00it/s, v_num=0]
Epoch 3: :

2023-09-07 09:09:47,979	INFO tune.py:1148 -- Total run time: 788.30 seconds (778.27 seconds for the tuning loop).
2023-09-07 09:09:47,980	WARNING tune.py:1158 -- Training has been interrupted, but the most recent state was saved.
Resume training with: Trainer.restore(path="/home/mpp/ray_results/ptl-sent-classification", ...)


Epoch 3: : 435it [02:24,  3.02it/s, v_num=0]


(LightningTrainer pid=18987) Traceback (most recent call last):
(LightningTrainer pid=18987)   File "python/ray/_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
(LightningTrainer pid=18987)   File "/home/mpp/.conda/envs/ray-torch/lib/python3.9/site-packages/ray/_private/function_manager.py", line 726, in actor_method_executor
(LightningTrainer pid=18987)     return method(__ray_actor, *args, **kwargs)
(LightningTrainer pid=18987)   File "/home/mpp/.conda/envs/ray-torch/lib/python3.9/site-packages/ray/util/tracing/tracing_helper.py", line 464, in _resume_span
(LightningTrainer pid=18987)     return method(self, *_args, **_kwargs)
(LightningTrainer pid=18987)   File "/home/mpp/.conda/envs/ray-torch/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 372, in train
(LightningTrainer pid=18987)     result = self.step()
(LightningTrainer pid=18987)   File "/home/mpp/.conda/envs/ray-torch/lib/python3.9/site-packages/ray/util/tracing/tracing_helper.py", lin

In [8]:
result

Result(
  metrics={'_report_on': 'validation_end', 'train_loss': 0.08543746918439865, 'matthews_correlation': 0.5930452712523209, 'epoch': 4, 'step': 2675, 'should_checkpoint': True, 'done': True, 'trial_id': '9082c_00000', 'experiment_tag': '0'},
  path='/home/dino/ray_results/ptl-sent-classification/LightningTrainer_9082c_00000_0_2023-09-06_23-51-03',
  checkpoint=LightningCheckpoint(local_path=/home/dino/ray_results/ptl-sent-classification/LightningTrainer_9082c_00000_0_2023-09-06_23-51-03/checkpoint_000004)
)